# Running a Federated Cycle with Synergos

This tutorial aims to give you an understanding of how to use the synergos package to run a full federated learning cycle. 

In a federated learning system, there are many contributory participants, known as Worker nodes, which receive a global model to train on, with their own local dataset. The dataset does not leave the individual Worker nodes at any point, and remains private to the node.

The job to synchronize, orchestrate and initiate an federated learning cycle, falls on a Trusted Third Party (TTP). The TTP pushes out the global model architecture and parameters for the individual nodes to train on, calling upon the required data, based on tags, e.g "training", which points to relevant data on the individual nodes. At no point does the TTP receive, copy or access the Worker nodes' local datasets.

In this tutorial, you will go through the steps required by each participant (TTP and Worker), by simulating each of them locally with docker containers. Specifically, we will simulate a TTP and 2 Workers. 

At the end of this, we will have:
- Connected the participants
- Trained the model
- Evaluate the model

## About the Dataset and Task

The dataset used in this notebook is on abalone age, comprising 8 predictor features, and 1 target feature. The dataset can be downloaded from [here](https://drive.google.com/drive/folders/15EpZaYr7gBYMKULZgpVTrJiDYZP0bQTv?usp=sharing). The task to be carried out will be a classification prediction.

## Initiating the docker containers

Before we begin, we have to start the docker containers.

Firstly, follow the instructions on [pysyft_ttp readme](https://gitlab.int.aisingapore.org/aims/federatedlearning/pysyft_worker/-/tree/dev) and [pysyft_worker readme](https://gitlab.int.aisingapore.org/aims/federatedlearning/pysyft_ttp/-/tree/plan_integrations) on how to build the required docker images for TTP and Workers.

Next, in <u>separate</u> CLI terminals, run the following command:

**Note: For Windows users, it is advisable to use powershell or command prompt based interfaces**

**Worker 1**

```docker run -v <directory abalone/data1>:/worker/data -v <directory abalone/outputs_1>:/worker/outputs --name worker_1 worker:pysyft_demo```

**Worker 2**

```docker run -v <directory abalone/data2>:/worker/data -v <directory abalone/outputs_2>:/worker/outputs --name worker_2 worker:pysyft_demo```

**TTP**

```docker run -p 0.0.0.0:5000:5000 -p 5678:5678 -p 8020:8020 -p 8080:8080 -v <directory abalone/mlflow_test>:/ttp/mlflow --name ttp --link worker_1 --link worker_2 ttp:pysyft_demo```

Once ready, for each terminal, you should see that a Flask app is running on http://0.0.0.0:5000 of the container.

You are now ready for the next step.

## Configuration

In a new terminal, run `docker inspect bridge` and find the IPv4Address for each container. Ideally, the containers should have the following addresses:
- worker_1 address: 172.17.0.2
- worker_2 address: 172.17.0.3
- ttp address: 172.17.0.4

If not, just note the relevant IP addresses for each docker container.

Run the following cells below.

**Note: For Windows users, `host` should be Docker Desktop VM's IP. Follow [this](https://stackoverflow.com/questions/58073936/how-to-get-ip-address-of-docker-desktop-vm) on instructions to find IP**

## Initializing Synergos driver

In [1]:
from synergos import Driver

host = "localhost"
port = 15000

driver = Driver(host=host, port=port)

## Phase 1: CONNECT - Submitting TTP & Participant metadata #

#### 1A. TTP controller creates a project

Create a project which is a classification task with action="classify", if the project is a regression task thenn action="regress".

In [2]:
# 1A. TTP controller creates a project
driver.projects.create(
    project_id="ks_project_1",
    action="regress",
    incentives={
        'tier_1': [],
        'tier_2': []
    }
)

{'data': {'doc_id': '2',
  'kind': 'Project',
  'key': {'project_id': 'ks_project_1'},
  'relations': {'Experiment': [],
   'Run': [],
   'Registration': [],
   'Tag': [],
   'Model': [],
   'Validation': [],
   'Prediction': []},
  'action': 'regress',
  'incentives': {'tier_2': [], 'tier_1': []}},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'projects.post',
 'params': {}}

#### 1B. TTP controller creates an experiment

In [3]:
# 1B. TTP controller creates an experiment
driver.experiments.create(
    project_id="ks_project_1",
    expt_id="ks_experiment_1",
    model=[
        {
            "activation": "",
            "is_input": True,
            "l_type": "Linear",
            "structure": {
                "bias": True,
                "in_features": 28,
                "out_features": 1
            }
        }
    ]
)

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'experiments.post',
 'params': {'project_id': 'ks_project_1'},
 'data': {'created_at': '2020-10-26 03:44:20 N',
  'key': {'expt_id': 'ks_experiment_1', 'project_id': 'ks_project_1'},
  'model': [{'activation': '',
    'is_input': True,
    'l_type': 'Linear',
    'structure': {'bias': True, 'in_features': 28, 'out_features': 1}}],
  'relations': {'Run': [], 'Model': [], 'Validation': [], 'Prediction': []},
  'doc_id': 2,
  'kind': 'Experiment'}}

#### 1C. TTP controller creates a run

In [4]:
driver.runs.create(
    project_id="ks_project_1",
    expt_id="ks_experiment_1",
    run_id="test_run",
    rounds=2, 
    epochs=1,
    base_lr=0.0005,
    max_lr=0.005,
    criterion="MSELoss"
)

{'data': {'doc_id': '2',
  'kind': 'Run',
  'key': {'project_id': 'ks_project_1',
   'expt_id': 'ks_experiment_1',
   'run_id': 'test_run'},
  'relations': {'Model': [{'doc_id': '1',
     'kind': 'Model',
     'key': {'project_id': 'ks_test_project_3',
      'expt_id': 'ks_test_experiment_3',
      'run_id': 'test_run'},
     'global': {'origin': 'ttp',
      'path': '/ttp/outputs/ks_test_project_3/ks_test_experiment_3/test_run/global_model.pt',
      'loss_history': '/ttp/outputs/ks_test_project_3/ks_test_experiment_3/test_run/global_loss_history.json'},
     'local_2': {'origin': 'test_participant_2',
      'path': '/ttp/outputs/ks_test_project_3/ks_test_experiment_3/test_run/local_model_test_participant_2.pt',
      'loss_history': '/ttp/outputs/ks_test_project_3/ks_test_experiment_3/test_run/local_loss_history_test_participant_2.json'},
     'local_1': {'origin': 'test_participant_1',
      'path': '/ttp/outputs/ks_test_project_3/ks_test_experiment_3/test_run/local_model_test_parti

#### 1D. Participants registers their servers' configurations

In [5]:
driver.participants.create(
    participant_id="test_participant_1",
    host='172.19.152.152',
#     host='172.17.0.2',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

driver.participants.create(
    participant_id="test_participant_2",
    host='172.19.152.153',
#     host='172.17.0.3',
    port=8020,
    f_port=5000,
    log_msgs=True,
    verbose=True
)

{'data': {'doc_id': '2',
  'kind': 'Participant',
  'key': {'participant_id': 'test_participant_2'},
  'relations': {'Registration': [{'doc_id': '2',
     'kind': 'Registration',
     'key': {'project_id': 'ks_test_project_3',
      'participant_id': 'test_participant_2'},
     'role': 'host'}],
   'Tag': [{'doc_id': '2',
     'kind': 'Tag',
     'key': {'project_id': 'ks_test_project_3',
      'participant_id': 'test_participant_2'},
     'train': [['train']],
     'evaluate': [],
     'predict': []}]},
  'id': 'test_participant_2',
  'host': '172.19.152.153',
  'port': 8020,
  'log_msgs': True,
  'verbose': True,
  'f_port': 5000},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'participants.post',
 'params': {}}

#### 1E. Participants registers their role in a specific project

In [6]:
driver.registrations.create(
    project_id="ks_project_1",
    participant_id="test_participant_1",
    role="guest"
)

driver.registrations.create(
    project_id="ks_project_1",
    participant_id="test_participant_2",
    role="host"
)

{'data': {'doc_id': '4',
  'kind': 'Registration',
  'key': {'project_id': 'ks_project_1',
   'participant_id': 'test_participant_2'},
  'relations': {'Tag': []},
  'project': {'incentives': {'tier_2': [], 'tier_1': []}, 'start_at': None},
  'participant': {'id': 'test_participant_2',
   'host': '172.19.152.153',
   'port': 8020,
   'f_port': 5000,
   'log_msgs': True,
   'verbose': True},
  'role': 'host'},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'registration.post',
 'params': {}}

#### 1F. Participants registers their tags for a specific project

In [7]:
driver.tags.create(
    project_id="ks_project_1",
    participant_id="test_participant_1",
    train=[
        ["train"]
    ],
    evaluate = [["evaluate"]]
)

driver.tags.create(
    project_id="ks_project_1",
    participant_id="test_participant_2",
    train=[["train"]]
)

{'data': {'doc_id': '4',
  'kind': 'Tag',
  'key': {'project_id': 'ks_project_1',
   'participant_id': 'test_participant_2'},
  'train': [['train']],
  'evaluate': [],
  'predict': []},
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'tags.post',
 'params': {'project_id': 'ks_project_1',
  'participant_id': 'test_participant_2'}}

## Phase 2: TRAIN - Alignment, Training & Optimisation

#### 2A. Perform multiple feature alignment to dynamically configure datasets and models for cross-grid compatibility

In [8]:
driver.alignments.create(project_id="ks_project_1")

{'apiVersion': '0.0.1',
 'success': 1,
 'status': 201,
 'method': 'alignments.post',
 'params': {'project_id': 'ks_project_1'},
 'data': [{'created_at': '2020-10-26 03:44:30 N',
   'key': {'participant_id': 'test_participant_2',
    'project_id': 'ks_project_1'},
   'link': {'alignment_id': '8dcd39fc173d11ebbf990242c0a80002',
    'registration_id': '8bb6d2ea173d11eb87750242c0a80002',
    'tag_id': '8bf9af16173d11ebba490242c0a80002'},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 3,
   'kind': 'Alignment'},
  {'created_at': '2020-10-26 03:44:30 N',
   'evaluate': {'X': [], 'y': []},
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'ks_project_1'},
   'link': {'alignment_id': '8dd003e4173d11ebbf990242c0a80002',
    'registration_id': '8baa129e173d11ebac740242c0a80002',
    'tag_id': '8bf3f77e173d11eb96dd0242c0a80002'},
   'train': {'X': [], 'y': []},
   'relations': {},
   'doc_id': 4,
   'kind': 'Alignment'}]}

#### 2B. Trigger training across the federated grid

In [9]:
model_resp = driver.models.create(
        project_id="ks_project_1",
        expt_id="ks_experiment_1",
        run_id="test_run")

## Phase 3: EVALUATE - Validation & Predictions

#### 3A. Perform validation(s) of combination(s)

In [10]:
driver.validations.create(
    project_id="ks_project_1",
    expt_id="ks_experiment_1",
    run_id="test_run"
)

{'data': [{'doc_id': '2',
   'kind': 'Validation',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'ks_project_1',
    'expt_id': 'ks_experiment_1',
    'run_id': 'test_run'},
   'evaluate': {'statistics': {'R2': -9.340906783872933,
     'MSE': 113.52977166662662,
     'MAE': 10.131801647087132},
    'res_path': '/worker/outputs/ks_project_1/ks_experiment_1/test_run/evaluate/inference_statistics_evaluate.json'}}],
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 200,
 'method': 'validations.post',
 'params': {'project_id': 'ks_project_1',
  'expt_id': 'ks_experiment_1',
  'run_id': 'test_run'}}

#### 3B. Perform prediction(s) of combination(s)

In [11]:
driver.predictions.create(
    tags={"ks_project_1": [["predict"]]},
    participant_id="test_participant_1",
    project_id="ks_project_1",
    expt_id="ks_experiment_1",
    run_id="test_run"
)

{'data': [{'doc_id': '1',
   'kind': 'Prediction',
   'key': {'participant_id': 'test_participant_1',
    'project_id': 'ks_project_1',
    'expt_id': 'ks_experiment_1',
    'run_id': 'test_run'},
   'predict': {'statistics': {'R2': -9.671762937010781,
     'MSE': 105.08107224651725,
     'MAE': 9.764873337827265},
    'res_path': '/worker/outputs/ks_project_1/ks_experiment_1/test_run/predict/inference_statistics_predict.json'}}],
 'apiVersion': '0.0.1',
 'success': 1,
 'status': 200,
 'method': 'predictions.post',
 'params': {'project_id': 'ks_project_1',
  'expt_id': 'ks_experiment_1',
  'run_id': 'test_run',
  'participant_id': 'test_participant_1'}}